## Expression Tutorial

In [ ]:
import hail as hl
hl.init()

### Hail's Data Types

Each object in Python has a data type, which can be accessed with the `type()` method. Here is a Python string, which has type `str`. 

In [ ]:
type("Python")

Hail has its own data types for representing data. Here is a Hail string, which we construct with the [str](https://hail.is/docs/devel/functions/core.html?highlight=str#hail.expr.functions.str) method. We can access the string's Hail type with the `dtype` field.

In [ ]:
hl.str("Hail").dtype

Hail has primitive and container types, as well as a few types specific to the field of genetics.

* primitive types: [int32](https://hail.is/docs/devel/types.html#hail.expr.types.tint32), [int64](https://hail.is/docs/devel/types.html#hail.expr.types.tint64), [float32](https://hail.is/docs/devel/types.html#hail.expr.types.tfloat32), [float64](https://hail.is/docs/devel/types.html#hail.expr.types.tfloat64), [bool](https://hail.is/docs/devel/types.html#hail.expr.types.tbool), [str](https://hail.is/docs/devel/types.html#hail.expr.types.tstr)
* container types: [arrays](https://hail.is/docs/devel/types.html#hail.expr.types.tarray), [sets](https://hail.is/docs/devel/types.html#hail.expr.types.tset), [dicts](https://hail.is/docs/devel/types.html#hail.expr.types.tdict), [tuples](https://hail.is/docs/devel/types.html#hail.expr.types.ttuple), [structs](https://hail.is/docs/devel/types.html#hail.expr.types.tstruct), [intervals](https://hail.is/docs/devel/types.html#hail.expr.types.tinterval)
* genetics types: [locus](https://hail.is/docs/devel/types.html#hail.expr.types.tlocus), [call](https://hail.is/docs/devel/types.html#hail.expr.types.tcall)

Hail data types are distinct from Python data types. A Hail string has a Hail type, `str`, but its Python type is a `StringExpression`. 

In [ ]:
hl.str("Hail")

In [ ]:
type(hl.str("Hail"))

### What is an Expression?

Data types in Hail are represented by [expression](https://hail.is/docs/devel/expressions.html#expressions) classes. Each data type has its own expression class. For example, an integer of type `tint32` is represented by an `Int32Expression`. 

We can construct an integer expression in Hail with the [int32](https://hail.is/docs/devel/functions/constructors.html?highlight=int32#hail.expr.functions.int32) function.

In [ ]:
hl.int32(3)

The `literal` function will convert a Python object into a Hail expression. Let's try it out on a Python list.

In [ ]:
hl.literal(['a', 'b', 'c'])

The Python list is converted to an ArrayExpression of type `array<str>`. In other words, an array of strings.

### Expressions are Lazy

In languages like Python and R, expressions are evaluated and stored immediately. This is called **eager** evalutation.

In [ ]:
1 + 2

Eager evaluation won't work on datasets that won't fit in memory. Consider the UK Biobank BGEN file, which is ~2TB but decompresses to >100TB in memory.

In order to process datasets of this size, Hail uses **lazy** evaluation. When you enter an expression, Hail doesn't execute the expression immediately: it simply records what you asked to do.

In [ ]:
one = hl.int32(1)
three = one + 2
three

Hail evaluates an expression only when it must. For example:

 - when performing an aggregation,
 - when calling `take`, `collect` or `show`,
 - when exporting or writing to disk.

Hail evaluates expressions by streaming to accomodate very large datasets.

If you want to force the evaluation of an expression, you can do so by calling `value`. Note that this can only be done on an expression with no index, such as `hl.int32(1) + 2`. If the expression has an index, e.g. `table.idx + 1`, 
then the `value` method will fail. The section on indices below explains this concept further. 

In [ ]:
three.value

The `show` method can also be used to evaluate and display the expression.

In [ ]:
three.show()

### Missing data

All expressions in Hail can represent missing data. Hail has a [collection of primitive operations](https://hail.is/docs/devel/functions/core.html) for dealing with missingness. 

The [null](https://hail.is/docs/devel/functions/core.html?highlight=null#hail.expr.functions.null) method can be used to create a missing expression of a specific type, such as a missing string:

In [ ]:
missing_string = hl.null(hl.tstr)

Expressions handle missingness in the following ways:

* a missing value plus another value is always missing
* a conditional statement with a missing predicate is missing
* when aggregating a sum of values, the missing values are ignored

This is different from Python's treatment of missingness, where `None + 5` would produce an error. In Hail, `hl.null(hl.tint32) + 5` produces a missing result, not an error. 

In [ ]:
hl.is_missing(hl.null(hl.tint32)+5).value

Here are a few more examples to illustrate how missingness is treated in Hail:

In [ ]:
hl.is_defined(hl.str("not missing")).value

In [ ]:
hl.sum(hl.array([1, 2, hl.null(hl.tint32)])).value

In [ ]:
hl.or_missing(True, hl.int32(5)).value

In [ ]:
print(hl.or_missing(False, hl.int32(5)).value)

### Indices

Expressions carry another piece of information: indices.  Indices record the `Table` or `MatrixTable` to which the expression refers, and the axes over which the expression can vary.

Let's see some examples from the 1000 genomes dataset:

In [ ]:
hl.utils.get_1kg('data/')

In [ ]:
mt = hl.read_matrix_table('data/1kg.mt')
mt

Let's add a global field.

In [ ]:
mt = mt.annotate_globals(dataset = '1kg')

We can examine any field of the matrix table with the `describe` method. If we examine the field we just added, notice that it has no indices, because it is a global field.

In [ ]:
mt.dataset.describe()

The `locus` field is a row field, so it will be indexed by `row`. 

In [ ]:
mt.locus.describe()

Likewise, a column field `s` will be indexed by `column`.

In [ ]:
mt.s.describe()

And finally, an entry field `GT` will be indexed by both the `row` and `column`.

In [ ]:
mt.GT.describe()

Expressions like `locus`, `s`, and `GT` above do not have a single value, but rather a value that varies across rows or columns of `mt`. Therefore, calling the `value` method on these expressions will lead to an error.

Global fields don't vary across rows or columns, so they have a `value`:

In [ ]:
mt.dataset.value

### `show`, `take`, and `collect`

Although expressions with indices have no `value`, you can use `show` to print the first few values, or `take` and `collect` to localize values to Python. 

`show` and `take` grab the first 10 rows by default, but you can specify a number of rows to grab.

In [ ]:
mt.s.show()

In [ ]:
mt.s.take(5)

You can `collect` an expression to localize all values, like getting a list of all sample IDs of a dataset.

But be careful -- don't `collect` more data than can fit in memory!

In [ ]:
all_sample_ids = mt.s.collect()
all_sample_ids[:5]

### Learning more

Hail has a suite of of [functions](https://hail.is/docs/devel/functions/index.html) to transform and build expressions.

Also, see the documentation for the [expressions](https://hail.is/docs/devel/expressions.html) themselves.